In [2]:
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceHub,Ollama
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage


PRELIMINARY_SUMMARY-
 1. PromptTemplate & ChatPromptTemplate uses same both execution techniques(i.e both model method & invoke) 
    - but slightly differs in construction of prompt(i.e prompt1=PromptTemplate() & prompt2=ChatPromptTemplate.from_message(list()) or .from_template()

2. **Ref-NOTE: https://python.langchain.com/v0.1/docs/use_cases/summarization/**
3. StuffDocumentaionChain
4."WITH CORRESPONDING CUSTOM CHUNK PROMPTS & COMBINED WHOLE DOC PROMPT "
5. MapReduce     USING map_prompt-chunk_prompt1 combined_prompt=whole_doc_prompt
6. Refine Method USING refine_prompt-chunk_prompt2 

# 1). Execution using PromptTemplate

In [3]:
llm1=Ollama(model="llama3:latest")

template1="provide brief overview on {text1}with word limit of {text2} and para to {text3}"
prompt1=PromptTemplate(input_variables=["text1", "text2", "text3"],template=template1)
llmchain1=LLMChain(llm=llm1,prompt=prompt1)


C:\Users\SANJEEV\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
input1,input2,input3=input("enter text1:"),input("enter text2:"),input("enter text3:")

#### Method-1=> model(prompt)

In [48]:
def myfunc(input1,input2,input3 ):

    llm1=Ollama(model="llama3:latest")

    template1="provide brief overview on {text1}with word limit of {text2} and para to {text3}"
    prompt1=PromptTemplate(input_variables=["text1", "text2", "text3"],template=template1)
    # llmchain1=LLMChain(llm=llm1,prompt=prompt1)
    x=llm1(prompt1.format(text1=input1,text2=input2,text3=input3))

    return x

# input2=input("enter text2:")
# input3=input("enter text3:")
output0=myfunc(input1,input2,input3)
output0

C:\Users\SANJEEV\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Here is a brief overview of Furious with a word count of 20:\n\nFurious is an American action comedy film directed by James Wan. It stars Vin Diesel, Michelle Rodriguez, Tyrese Gibson, Chris "Ludacris" Bridges, Matt Gallant, Sung Kang, Gal Gadot, and Dwayne Johnson. The movie follows Dominic Toretto (Diesel) as he reunites with his crew to take down a ruthless businessman, who has targeted them for revenge.'

#### Method-2 Using LLMChain or chain | operator

##### a) using llmchain--might soon be deprecated but same formati.e (Input dictionary(containing multiple Key-Value pairs) is used to to call multiple inputs

In [6]:
#  llmchain1=LLMChain(llm=llm1,prompt=prompt1)

# input1=input("enter input1")
# input2=input("enter input2")
# input3=input("enter input3")
"""https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain"""
"""for parameter inputs-Dictionary of inputs, or single input if chain expects only one param. Should contain all inputs specified in Chain.input_keys except for inputs that will be set by the chain’s memory."""
output1=llmchain1(inputs={"text1":input1,"text2":input2,"text3":input3})
output1

C:\Users\SANJEEV\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'text1': 'furious',
 'text2': '20',
 'text3': '1',
 'text': 'Here is a brief overview of Furious with a 20-word limit and one paragraph:\n\n**Overview:** Fast-paced action film about Dominic Toretto (Vin Diesel) and his crew\'s high-stakes heists.\n\n"Ride or die," the Furious franchise motto, sums up the adrenaline-fueled adventures of Dom and his loyal crew as they outrun and outgun their foes in a series of high-octane heists.'}

##### b) using CHAIN | Operator(As Replacement to a) )
- Holds same format for input parsing as above i.e (Input dictionary(containing multiple Key-Value pairs)) is used to to call multiple inputs
where invoke implicitly calls inputs dictionary to call its input key-value pairs) 


In [7]:
alt_chain=prompt1 |llm1
alt_chain

PromptTemplate(input_variables=['text1', 'text2', 'text3'], template='provide brief overview on {text1}with word limit of {text2} and para to {text3}')
| Ollama(model='llama3:latest')

In [13]:
output2=alt_chain.invoke({"text1":input1,"text2":input2,"text3":input3})
output2

'Here is a brief overview of Furious:\n\n**Overview (20 words)**\nFurious: Fast & Furious franchise film, action-packed thriller with high-octane racing, heists, and family.'

NOTE:Number of tokens in output2 using the model  & can be perceived as prerequisite feature to LIMIT THE MAX TOKENS IF oN tOKEN CRUNCHFOR THE CURRENT BILLING PERIOD

### Get the number of tokens in the output using the model

In [14]:
llm1.get_num_tokens(output2)

44

In [53]:
type(output2)

str

In [15]:
new_input="translate "+output2+" to {lang} "
prompt2=PromptTemplate(input_variables=["lang","pronun_lang"],template=new_input)
input11=input("enter translational language to be converted into ")
# input22=input("enter pronunciation_lang")
llm1(prompt2.format(lang=input11))

'Here\'s the translation:\n\n**ஒரு குறிப்பு (20 செய்தி)**\n\nஃஃ: Fast & Furious வழக்குறி, அதிருட்சி நிகண்ட் திரைப்படம், உயர்ந்த-ஒக்ட்னே ஓட்டப்பாடியும், ஏட்சும் மற்றும் குடும்பங்கள்.\n\nNote: I\'ve translated "high-octane" to "உயர்ந்த-ஒக்ட்னே" which is a direct translation of the phrase, but it may not be as idiomatic in Tamil. A more natural translation would be something like "உயர்ந்த விரிச்சி" or "அதிருட்சி ஓட்டப்பாடியும்". Let me know if you\'d like me to make any changes!'

# 2) Execution using ChatPromptTemplate

In [ ]:
input21=input("enter the topic of conversation")
input22=input("enter word limit")

- NOTE:- by default all input message fromat are enclosed in list except (.from _template version)

#### a)Using Model method-1 to retrieve response

##### a1.) Using Default Baseline message format

In [83]:
chat_prompt1_a=ChatPromptTemplate.from_messages([HumanMessage(content="provide brief overview on {text21}with word limit of {text22} .")
                                              ,SystemMessage(content="Nehave as Ai Sonnet assistant ")])

llm1(chat_prompt1_a.format(text21=input21,text22=input22))

"I'm happy to help! However, I need more information from you.\n\nPlease provide the text ({text21}) and the word limit ({text22}), and I'll be delighted to assist you in crafting a brief overview within that limit.\n\nRemember, I'm your AI Sonnet assistant, here to help you shine with well-crafted content!"

##### a2.) Using Mixed message format

In [70]:
chat_prompt1_b=ChatPromptTemplate.from_messages([("human","provide brief overview on {text21}with word limit of {text22} ."),
                                              SystemMessage(content="Behave as AI Sonnet assistant ")])

llm1(chat_prompt1_b.format(text21=input21,text22=input22))

"A gun, a tool for hunting and defense,\nWith metal and fire, it can take its offense.\nA device for precision, or so they claim,\nBut often used for harm, causing endless pain.\n\n(Note: I've kept the overview within the 10-word limit as per your request!)"

##### a3.) Using Default tuple based message format

In [74]:
chat_prompt1_c=ChatPromptTemplate.from_messages([("human","provide brief overview on {text21}with word limit of {text22} ."),
                                              ("system","Behave as AI Sonnet assistant ")])

llm1(chat_prompt1_c.format(text21=input21,text22=input22))

'A request for a brief overview on guns!\n\nWithin the word limit of 10 words, here\'s my attempt:\n\n"Guns are firearms designed for hunting, self-defense, or sport."'

##### a4.) Using TEMPLATE Based message format(by Default assumes input from Human Message)

In [80]:
chat_prompt1_d=ChatPromptTemplate.from_template("provide brief overview on {text21}with word limit of {text22} .")

llm1(chat_prompt1_d.format(text21=input21,text22=input22))

'Here\'s a brief overview of guns within the 10-word limit:\n\n"A firearm is a portable device for shooting projectiles."'

#### b) Using INVOKE method-2 to retrieve response

##### b1.) Using Default Baseline message format

In [82]:
chat_prompt2_a=ChatPromptTemplate.from_messages([HumanMessage(content="provide brief overview on {text21}with word limit of {text22} .")
                                              ,SystemMessage(content="Nehave as Ai Sonnet assistant ")])
chain_chat1_prompt_a=chat_prompt2_a |llm1

chain_chat1_prompt_a.invoke({"text21":input21,"text22":input22})

"I'm happy to help! However, I need more information from you. It seems like you want me to provide a brief overview of some text, but you haven't specified what that text is.\n\nCould you please provide the text you'd like me to summarize, and also specify the word limit for the summary? Once I have that information, I'll be happy to assist you!"

##### b2.) Using Mixed message format

In [86]:
chat_prompt2_b=ChatPromptTemplate.from_messages([("human","provide brief overview on {text21}with word limit of {text22} ."),
                                              SystemMessage(content="Behave as AI Sonnet assistant ")])
chain_chat1_prompt_b=chat_prompt2_b |llm1

chain_chat1_prompt_b.invoke({"text21":input21,"text22":input22})

"A gun, a tool of might,\nFirearms for defense or delight.\n\n(Note: Within the 10-word limit, I've provided a brief overview of a gun)"

##### b3.) Using Default tuple based message format

In [87]:
chat_prompt2_c=ChatPromptTemplate.from_messages([("human","provide brief overview on {text21}with word limit of {text22} ."),
                                              ("system","Behave as AI Sonnet assistant ")])
chain_chat1_prompt_c=chat_prompt2_c |llm1

chain_chat1_prompt_c.invoke({"text21":input21,"text22":input22})

'Here\'s a brief overview of guns within the 10-word limit:\n\n"Guns are firearms designed for hunting, sport, or self-defense purposes."'

##### b4.) Using TEMPLATE Based message format(by Default assumes input from Human Message)

In [88]:
chat_prompt1_d=ChatPromptTemplate.from_template("provide brief overview on {text21}with word limit of {text22} .")
chain_chat1_prompt_d=chat_prompt1_d |llm1

chain_chat1_prompt_d.invoke({"text21":input21,"text22":input22})

'Here\'s a brief overview of a gun in 10 words:\n\n"A firearm, typically made of metal and wood, for hunting/shooting."'

# 3). StuffDocumentaionChain

In [5]:
from PyPDF2 import PdfReader

In [6]:
pdf_read=PdfReader("G:\CORE-3\OPEN_AI_1\TEXT_SUMMARIZATION_IMPLEMENTATION\Monolith_Real Time Recommendation System_BD.pdf")

In [7]:
"""For visualization pupose to user - Not implemented down the lane"""
content_of_pdf=""
for i,page in enumerate(pdf_read.pages):
    text_extracted=page.extract_text()
    if(text_extracted):
        content_of_pdf+=text_extracted
content_of_pdf

'Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental to model quality; on the other hand, such\n

In [8]:
from langchain_community.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

In [9]:
doc_store=[Document(page_content=content_of_pdf)]
doc_store

[Document(page_content='Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental to model quality; on

#####  IMP-NOTE:-Below the stuff documentation chain  expects variable {"text"} in its Prompt Template rather than other randome input variable for parsing the given doc store as an input while invoking.

In [14]:
template2="Provide summary of the above pdf {text}"
prompt_doc=PromptTemplate(input_variables=["text"],template=template2)
# Chain_doc=load_summarize_chain(llm=llm1, prompt=prompt_doc,chain_type="stuff")
# Chain_doc(doc_store)

In [37]:
Chain_doc=load_summarize_chain(llm=llm1, chain_type="stuff", prompt=prompt_doc,verbose=True)
"""ValueError: Missing some input keys: {'input_documents'}"""
Chain_doc.invoke(doc_store)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Provide summary of the above pdf Monolith: Real Time Recommendation System With
Collisionless Embedding Table
Zhuoran Liu
Bytedance Inc.Leqi Zou
Bytedance Inc.Xuan Zou
Bytedance Inc.
Caihua Wang
Bytedance Inc.Biao Zhang
Bytedance Inc.Da Tang
Bytedance Inc.
Bolin Zhu∗
Fudan UniversityYijie Zhu
Bytedance Inc.Peng Wu
Bytedance Inc.
Ke Wang
Bytedance Inc.Youlong Cheng†
Bytedance Inc.
youlong.cheng@bytedance.com
ABSTRACT
Building a scalable and real-time recommendation system is vital
for many businesses driven by time-sensitive customer feedback,
such as short-videos ranking or online ads. Despite the ubiquitous
adoption of production-scale deep learning frameworks like Ten-
sorFlow or PyTorch, these general-purpose frameworks fall short
of business demands in recommendation scenarios for various rea-
sons: on one hand, tweaking systems based on static parameters and
dense computations

{'input_documents': [Document(page_content='Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental 

# 4) MAP_REDUCE METHOD 
- (Overcomes drawback of Stuff Documentation Chain to parse Large pdf files that has TOken-Restriction set at model)
- i.e if document is large to be parsed & tokens of summarization exceeds the maximum number of tokens set in the model, then we can use MAP_REDUCE method to split the large document into multiple chunks, thereby minimizing errors &parsing the output chunks until it hits the max_tokens set by model. 


####  Note:Addtional Lnagchain module to split the large documents into chunks

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

Utlizing above content_of_pdf as text to create multiple chunks.

In [39]:
content_of_pdf

'Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental to model quality; on the other hand, such\n

In [43]:
Text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap= 25)
doc_chunks=Text_splitter.create_documents([content_of_pdf])
doc_chunks

[Document(page_content='Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental to model quality; on

In [45]:
len(doc_chunks)

44

In [50]:
doc_chunks[:2]

[Document(page_content='Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental to model quality; on

In [56]:
doc_template3="Provide summary of the above pdf {text}"
prompt_doc3=PromptTemplate(input_variables=["text"],template=template3)
Chain_doc=load_summarize_chain(llm=llm1, chain_type="map_reduce",combine_prompt=prompt_doc3,verbose=True)
"""ValueError: extra value(s) found in load_summarize_chain: {prompt=prompt_doc3} -- therefore should be ommitted since By defualt the the funcion implies summarizing of the doc chunk provided"""
"""Limited the no of chunks(not of dos's pages to 5 in interest of time for model to parse the summary)"""
Chain_doc.invoke(doc_chunks[:5])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Monolith: Real Time Recommendation System With
Collisionless Embedding Table
Zhuoran Liu
Bytedance Inc.Leqi Zou
Bytedance Inc.Xuan Zou
Bytedance Inc.
Caihua Wang
Bytedance Inc.Biao Zhang
Bytedance Inc.Da Tang
Bytedance Inc.
Bolin Zhu∗
Fudan UniversityYijie Zhu
Bytedance Inc.Peng Wu
Bytedance Inc.
Ke Wang
Bytedance Inc.Youlong Cheng†
Bytedance Inc.
youlong.cheng@bytedance.com
ABSTRACT
Building a scalable and real-time recommendation system is vital
for many businesses driven by time-sensitive customer feedback,
such as short-videos ranking or online ads. Despite the ubiquitous
adoption of production-scale deep learning frameworks like Ten-
sorFlow or PyTorch, these general-purpose frameworks fall short
of business demands in recommendation scenarios for various rea-
sons: on one hand, tweaking systems based on static parameters and
de

{'input_documents': [Document(page_content='Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental 

# 5) MAP_REDUCE METHOD WITH Custom Prompt for both (Chunk & whole document) 
-- **NOTE-1:using (map_prompt=chunk-prompt1)& (combine_prompt=prompt_doc3). Where each of the above can be used isolation with each other**

In [63]:
"""chunks prompt created"""
chunk_template1="Provide summary of the paper {text} until now"
chunk_prompt1=PromptTemplate(input_variables=["text"],template=chunk_template1)

""""whole_doc prompt"""
doc_template4="Provide information in bulleted list format for each of the above chunk summaries from pdf {text}"
prompt_doc4=PromptTemplate(input_variables=["text"],template=doc_template4)

Chain_doc2=load_summarize_chain(llm=llm1, chain_type="map_reduce",map_prompt=chunk_prompt1,combine_prompt=prompt_doc4,verbose=True)
"""ValueError: extra value(s) found in load_summarize_chain: {prompt=prompt_doc3} -- therefore should be ommitted since By defualt the the funcion implies summarizing of the doc chunk provided"""
"""Limited the no of chunks(not of dos's pages to 5 in interest of time for model to parse the summary)"""
Chain_doc2.invoke(doc_chunks[:5])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Provide summary of the paper Monolith: Real Time Recommendation System With
Collisionless Embedding Table
Zhuoran Liu
Bytedance Inc.Leqi Zou
Bytedance Inc.Xuan Zou
Bytedance Inc.
Caihua Wang
Bytedance Inc.Biao Zhang
Bytedance Inc.Da Tang
Bytedance Inc.
Bolin Zhu∗
Fudan UniversityYijie Zhu
Bytedance Inc.Peng Wu
Bytedance Inc.
Ke Wang
Bytedance Inc.Youlong Cheng†
Bytedance Inc.
youlong.cheng@bytedance.com
ABSTRACT
Building a scalable and real-time recommendation system is vital
for many businesses driven by time-sensitive customer feedback,
such as short-videos ranking or online ads. Despite the ubiquitous
adoption of production-scale deep learning frameworks like Ten-
sorFlow or PyTorch, these general-purpose frameworks fall short
of business demands in recommendation scenarios for various rea-
sons: on one hand, tweaking systems based on static parameters and
dense computations

{'input_documents': [Document(page_content='Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental 

# 6) REFINE METHOD WITH Custom Prompt for both (Chunk & whole document)  [|||AR TO MAP_REDUCE METHOD]
- **BUT EACH INPUT CHUNK IS PASSED TO SUBSEQUENT I/PS IN CDF FASHION(CUMULATIVE DISTRIBUTION FUNCTION) OR CUMULATIVE SUM**
- **NOTE-2:using (refine_prompt=chunk-prompt2)& (question_prompt=prompt_doc5/whole_doc_prompt5). Where each of the above can be used isolation with each other**

In [70]:
"""chunks prompt created"""
chunk_template2="Provide summary of the paper {text} until now"
chunk_prompt2=PromptTemplate(input_variables=["text"],template=chunk_template2)

""""whole_doc prompt"""
doc_template5="Provide information in bulleted list format for each of the above chunk summaries from pdf {text}"
prompt_doc5=PromptTemplate(input_variables=["text"],template=doc_template5)

Chain_doc3=load_summarize_chain(llm=llm1, chain_type="refine",refine_prompt=chunk_prompt2,question_prompt=prompt_doc5,verbose=True)
"""ValueError: extra value(s) found in load_summarize_chain: {prompt=prompt_doc3} -- therefore should be ommitted since By defualt the the funcion implies summarizing of the doc chunk provided"""
"""Limited the no of chunks(not of dos's pages to 5 in interest of time for model to parse the summary)"""
Chain_doc3.invoke(doc_chunks[:5])



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Provide information in bulleted list format for each of the above chunk summaries from pdf Monolith: Real Time Recommendation System With
Collisionless Embedding Table
Zhuoran Liu
Bytedance Inc.Leqi Zou
Bytedance Inc.Xuan Zou
Bytedance Inc.
Caihua Wang
Bytedance Inc.Biao Zhang
Bytedance Inc.Da Tang
Bytedance Inc.
Bolin Zhu∗
Fudan UniversityYijie Zhu
Bytedance Inc.Peng Wu
Bytedance Inc.
Ke Wang
Bytedance Inc.Youlong Cheng†
Bytedance Inc.
youlong.cheng@bytedance.com
ABSTRACT
Building a scalable and real-time recommendation system is vital
for many businesses driven by time-sensitive customer feedback,
such as short-videos ranking or online ads. Despite the ubiquitous
adoption of production-scale deep learning frameworks like Ten-
sorFlow or PyTorch, these general-purpose frameworks fall short
of business demands in recommendation scenarios for various rea-
sons: on one hand, tweakin


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide summary of the paper frameworks are designed with batch-training stage and serving
stage completely separated, preventing the model from interacting
with customer feedback in real-time. These issues led us to reex-
amine traditional approaches and explore radically different design
choices. In this paper, we present Monolith1, a system tailored
for online training. Our design has been driven by observations
of our application workloads and production environment that re-
flects a marked departure from other recommendations systems.
Our contributions are manifold: first, we crafted a collisionless em-
bedding table with optimizations such as expirable embeddings
and frequency filtering to reduce its memory footprint; second, we
provide an production-ready online training architecture with high
fault-tolerance; finally, we proved that system reliability could be
traded-off for real-time learning. Monol

{'input_documents': [Document(page_content='Monolith: Real Time Recommendation System With\nCollisionless Embedding Table\nZhuoran Liu\nBytedance Inc.Leqi Zou\nBytedance Inc.Xuan Zou\nBytedance Inc.\nCaihua Wang\nBytedance Inc.Biao Zhang\nBytedance Inc.Da Tang\nBytedance Inc.\nBolin Zhu∗\nFudan UniversityYijie Zhu\nBytedance Inc.Peng Wu\nBytedance Inc.\nKe Wang\nBytedance Inc.Youlong Cheng†\nBytedance Inc.\nyoulong.cheng@bytedance.com\nABSTRACT\nBuilding a scalable and real-time recommendation system is vital\nfor many businesses driven by time-sensitive customer feedback,\nsuch as short-videos ranking or online ads. Despite the ubiquitous\nadoption of production-scale deep learning frameworks like Ten-\nsorFlow or PyTorch, these general-purpose frameworks fall short\nof business demands in recommendation scenarios for various rea-\nsons: on one hand, tweaking systems based on static parameters and\ndense computations for recommendation with dynamic and sparse\nfeatures is detrimental 